## Userful links:
https://docs.opencv.org/3.4/d7/dff/tutorial_feature_homography.html  
https://www.youtube.com/playlist?list=PLMoSUbG1Q_r-JNMQ0zJmv6SnXwgbA8JJp  

In [31]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import math
import os
import mpl_toolkits.mplot3d.axes3d as p3
import plotly.graph_objects as go

%matplotlib inline

In [32]:
def ResizeWithAspectRatio(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]

    if width is None and height is None:
        return image
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    else:
        r = width / float(w)
        dim = (width, int(h * r))

    return cv2.resize(image, dim, interpolation=inter)

## Pattern initialization

In [33]:
pattern = ResizeWithAspectRatio(cv2.imread('patterns/chessBoard_9x7.png'), width=720)
patternGray = cv2.cvtColor(pattern, cv2.COLOR_BGR2GRAY)
H, W, _ = pattern.shape

video = cv2.VideoCapture('videos/fractals.mp4')
_, imgVideo = video.read()
imgVideo = cv2.resize(imgVideo, (W, H))

## SIFT keypoints detector

In [5]:
#sift keypoints visualize

cam = cv2.VideoCapture(0)

sift = cv2.SIFT_create()

while(cam.isOpened()):
    ret, frame = cam.read()
    frame = ResizeWithAspectRatio(frame, width=720)
    if ret:
        (keypoints, descriptors) = sift.detectAndCompute(frame, None)
        for kp in keypoints:
            x, y = kp.pt
            x = int(round(x))
            y = int(round(y))
            cv2.drawMarker(frame, (x, y), (255, 255, 255), markerType=cv2.MARKER_CROSS, markerSize=5, thickness=1, line_type=cv2.LINE_8)
        cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q') or cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()
cam.release()

## ORB keypoints descriptor

In [6]:
#ORB algorithm

cam = cv2.VideoCapture(0)

orb = cv2.ORB_create(nfeatures=1000, fastThreshold=0, edgeThreshold=0)
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

kp1, des1 = orb.detectAndCompute(patternGray, None)
detect = False
framesCount = 0

while(cam.isOpened()):
    ret, frame = cam.read()
    imgAug = frame.copy()
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        kp2, des2 = orb.detectAndCompute(gray, None)
        
        if detect == False:
            video.set(cv2.CAP_PROP_POS_FRAMES,0)
            framesCount = 0
        else:
            if framesCount == video.get(cv2.CAP_PROP_FRAME_COUNT):
                video.set(cv2.CAP_PROP_POS_FRAMES, 0)
                framesCount = 0
            _, imgVideo = video.read()
            imgVideo = cv2.resize(imgVideo, (W, H))
        
        if des1 is not None and des2 is not None:
            matches = bf.match(des1, des2)
            matches = sorted(matches, key = lambda x:x.distance)
            good = []

            for k in range(0, min(50, len(matches))):
                m = matches[k]
                if m.distance < 30:
                    good.append(m)

            if len(good) > 40:
                detect = True
                srcPts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
                dstPts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

                matrix, _ = cv2.findHomography(srcPts, dstPts, cv2.RANSAC, 5.0)
                
                if matrix is None:
                    framesCount +=1
                    continue
                pts = np.float32([[0, 0], [0, H-1], [W-1, H-1], [W-1, 0]]).reshape(-1, 1, 2)

                dst = cv2.perspectiveTransform(pts, matrix)
                imgPol = cv2.polylines(frame, [np.int32(dstPts)], True, (255, 0, 255), 3)

                imgWarp = cv2.warpPerspective(imgVideo, matrix, (frame.shape[1], frame.shape[0]))

                maskNew = np.zeros((frame.shape[0], frame.shape[1]), np.uint8)
                
                cv2.fillPoly(maskNew, [np.int32(dst)], (255, 255, 255))
                maskInv = cv2.bitwise_not(maskNew)
                imgAug = cv2.bitwise_and(imgAug, imgAug, mask = maskInv)
                imgAug = cv2.bitwise_or(imgWarp, imgAug)

                match = cv2.drawMatches(pattern, kp1, frame, kp2, good, None, flags=0)
                #cv2.imshow('frame', frame)
                cv2.imshow('match', match)
                #cv2.imshow('imgPol', imgPol)
                #cv2.imshow('aug', imgAug)
                cv2.imshow('warp', imgWarp)
                #cv2.imshow('imgVideo', imgVideo)
            else:
                cv2.imshow('frame', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q') or cv2.waitKey(1) & 0xFF == 27:
            break
    else:
        break
    
    framesCount +=1

cv2.destroyAllWindows()
cam.release()

## SIFT keypoints descriptor

In [21]:
#SIFT algorithm

cam = cv2.VideoCapture(0)

sift = cv2.SIFT_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=False)

FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks = 50)

flann = cv2.FlannBasedMatcher(index_params, search_params)

kp1, des1 = sift.detectAndCompute(patternGray, None)


detect = False
framesCount = 0

while(cam.isOpened()):
    ret, frame = cam.read()
    imgAug = frame.copy()
    if ret:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        kp2, des2 = sift.detectAndCompute(gray, None)
        
        if detect == False:
            video.set(cv2.CAP_PROP_POS_FRAMES,0)
            framesCount = 0
        else:
            if framesCount == video.get(cv2.CAP_PROP_FRAME_COUNT):
                video.set(cv2.CAP_PROP_POS_FRAMES, 0)
                framesCount = 0
            _, imgVideo = video.read()
            imgVideo = cv2.resize(imgVideo, (W, H))
        
        if des1 is not None and des2 is not None:
            matches = flann.knnMatch(des1, des2, k=2)
            
            good = []
            for m, n in matches:
                if m.distance < 0.8*n.distance:
                    good.append(m)
            
            if len(good) > 20:
                detect = True
                srcPts = np.float32([kp1[m.queryIdx].pt for m in good]).reshape(-1, 1, 2)
                dstPts = np.float32([kp2[m.trainIdx].pt for m in good]).reshape(-1, 1, 2)

                matrix, mask = cv2.findHomography(srcPts, dstPts, cv2.RANSAC, 5.0)
                matchesMask = mask.ravel().tolist()
                
                if matrix is None:
                    framesCount +=1
                    continue
                pts = np.float32([[0, 0], [0, H-1], [W-1, H-1], [W-1, 0]]).reshape(-1, 1, 2)

                dst = cv2.perspectiveTransform(pts, matrix)
                imgPol = cv2.polylines(frame, [np.int32(dstPts)], True, (255, 0, 255), 3)

                imgWarp = cv2.warpPerspective(imgVideo, matrix, (frame.shape[1], frame.shape[0]))

                maskNew = np.zeros((frame.shape[0], frame.shape[1]), np.uint8)
                
                cv2.fillPoly(maskNew, [np.int32(dst)], (255, 255, 255))
                maskInv = cv2.bitwise_not(maskNew)
                imgAug = cv2.bitwise_and(imgAug, imgAug, mask = maskInv)
                imgAug = cv2.bitwise_or(imgWarp, imgAug)
                
                draw_params = dict(matchColor = (255, 0, 0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)

                match = cv2.drawMatches(pattern, kp1, frame, kp2, good, None, **draw_params)
                #cv2.imshow('frame', frame)
                cv2.imshow('match', match)
                #cv2.imshow('imgPol', imgPol)
                #cv2.imshow('aug', imgAug)
                #cv2.imshow('warp', imgWarp)
                #cv2.imshow('imgVideo', imgVideo)
            else:
                cv2.imshow('frame', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q') or cv2.waitKey(1) & 0xFF == 27:
            break
    else:
        break
    
    framesCount +=1

cv2.destroyAllWindows()
cam.release()

## Chess board pattern recognize

In [49]:
#Chess board pattern algorithm

nX = 10
nY = 7

cam = cv2.VideoCapture(0)
ret1, kp1 = cv2.findChessboardCorners(patternGray, (nX-1, nY-1), None)
if ret1:
    detect = False
    framesCount = 0

    while(cam.isOpened()):
        ret, frame = cam.read()
        imgAug = frame.copy()
        if ret:
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            ret2, kp2 = cv2.findChessboardCorners(gray, (nX-1, nY-1), None,
                            cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE + cv2.CALIB_CB_FAST_CHECK)

            if detect == False:
                video.set(cv2.CAP_PROP_POS_FRAMES, 0)
                framesCount = 0
            else:
                if framesCount == video.get(cv2.CAP_PROP_FRAME_COUNT):
                    video.set(cv2.CAP_PROP_POS_FRAMES, 0)
                    framesCount = 0
                _, imgVideo = video.read()
                imgVideo = cv2.resize(imgVideo, (W, H))
            
            if ret2:
                detect = True
                
                matrix, _ = cv2.findHomography(kp1, kp2, cv2.RANSAC, 5.0)
                
                if matrix is None:
                    framesCount += 1
                    continue
                pts = np.float32([[0, 0], [0, H-1], [W-1, H-1], [W-1, 0]]).reshape(-1, 1, 2)

                dst = cv2.perspectiveTransform(pts, matrix)

                imgWarp = cv2.warpPerspective(imgVideo, matrix, (frame.shape[1], frame.shape[0]))

                maskNew = np.zeros((frame.shape[0], frame.shape[1]), np.uint8)
                
                cv2.fillPoly(maskNew, [np.int32(dst)], (255, 255, 255))
                maskInv = cv2.bitwise_not(maskNew)
                imgAug = cv2.bitwise_and(imgAug, imgAug, mask = maskInv)
                imgAug = cv2.bitwise_or(imgWarp, imgAug)
                
                cv2.imshow('frame', imgAug)
            else:
                cv2.imshow('frame', frame)
            

            if cv2.waitKey(1) & 0xFF == ord('q') or cv2.waitKey(1) & 0xFF == 27:
                break
        else:
            break

        framesCount +=1

    cv2.destroyAllWindows()
    cam.release()
else:
    print("Chess board pattern not found")